## KoGPT-2로 문장 생성하기

transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT와 마찬가지로 GPT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [1]:
import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

TFGPT2LMHeadModel.from_pretrained('GPT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 GPT 구조를 로드합니다. AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [3]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
sent = '오늘 날씨가'

In [6]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[10070 34018]], shape=(1, 2), dtype=int32)


In [9]:
output = model.generate(input_ids,
                        max_length=20,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[10070, 34018, 9220, 8102, 12762, 387, 11457, 8063, 8146, 13964, 7965, 9810, 10590, 30899, 17339, 9563, 8135, 9097, 8702, 9130]


In [10]:
tokenizer.decode(output_ids)

'오늘 날씨가 추워지면서, 겨울옷이 얇아지고 있습니다.\n오늘 아침 서울은 영하 4'

## Numpy로 Top 5 뽑기

In [11]:
output = model(input_ids)
top5 = tf.math.top_k(output.logits[0, -1], k=5)

In [12]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy())

['▁추', '▁쌀', '▁좋', '▁더', '▁많이']

## Numpy Top 5로 문장 생성하기

In [14]:
sent = '오늘 날씨가'
input_ids = tokenizer.encode(sent)

while len(input_ids) < 20:
    output = model(np.array([input_ids]))
    # Top 5의 단어들을 추출
    top5 = tf.math.top_k(output.logits[0, -1], k=10)
    # Top 5의 단어들 중 랜덤으로 다음 단어로 선택.
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

tokenizer.decode(input_ids)

'오늘 날씨가 추워진다고 해도 이 맘땐가 싶었다.\n지루한 마음을 가라앉히려면 더'